# Word category formation using BERT predictions

## The idea is to generate word embeddings by:
- Get a list of sentences
- Mask a word in each sentence (repeat a sentence in the list if you want to mask different positions)
- For each sentence, obtain the logit vector for the masked word from BERT's prediction (last hidden layer)
- Cluster sentences logit vectors. The clusters should reflect words that fit together both syntactically and semantically.
- Build each word category by finding the highest valued words in the vectors belonging to a cluster (perhaps by most common top words, all words above some threshold, etc)

In [1]:
from transformers import BertTokenizer, BertForMaskedLM
import numpy as np
import torch
import re

In [2]:
with torch.no_grad():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertForMaskedLM.from_pretrained('bert-base-uncased')
    model.eval()

### Choose some simple sentences with masked adjectives and nouns

In [3]:
text_sentences = """_ fat cat ate the mouse.
The _ cat ate the mouse.
The fat _ ate the mouse.
The fat cat _ the mouse.
The fat cat ate _ mouse.
The fat cat ate the _.
_ was wearing a lovely satin dress last night.
She _ wearing a lovely satin dress last night.
She was _ a lovely satin dress last night.
She was wearing _ lovely satin dress last night.
She was wearing a _ satin dress last night.
She was wearing a lovely _ dress last night.
She was wearing a lovely satin _ last night.
She was wearing a lovely satin dress _ night.
She was wearing a lovely satin dress last _.
_ was receiving quite a hefty salary.
He _ receiving quite a hefty salary.
He was _ quite a hefty salary.
He was receiving _ a hefty salary.
He was receiving quite _ hefty salary.
He was receiving quite a _ salary.
He was receiving quite a hefty _.
_ also bought a used sofa for his new apartment.
He _ bought a used sofa for his new apartment.
He also _ a used sofa for his new apartment.
He also bought _ used sofa for his new apartment.
He also bought a _ sofa for his new apartment.
He also bought a used _ for his new apartment.
He also bought a used sofa _ his new apartment.
He also bought a used sofa for _ new apartment.
He also bought a used sofa for his _ apartment.
He also bought a used sofa for his new _.
_ was born and grew up in Havana.
I _ born and grew up in Havana.
I was _ and grew up in Havana.
I was born _ grew up in Havana.
I was born and _ up in Havana.
I was born and grew _ in Havana.
I was born and grew up _ Havana.
I was born and grew up in _.
_ Beijing metropolitan area added more than a million people in the past decade.
The _ metropolitan area added more than a million people in the past decade.
The Beijing _ area added more than a million people in the past decade.
The Beijing metropolitan _ added more than a million people in the past decade.
The Beijing metropolitan area _ more than a million people in the past decade.
The Beijing metropolitan area added _ than a million people in the past decade.
The Beijing metropolitan area added more _ a million people in the past decade.
The Beijing metropolitan area added more than _ million people in the past decade.
The Beijing metropolitan area added more than a _ people in the past decade.
The Beijing metropolitan area added more than a million _ in the past decade.
The Beijing metropolitan area added more than a million people _ the past decade.
The Beijing metropolitan area added more than a million people in _ past decade.
The Beijing metropolitan area added more than a million people in the _ decade.
The Beijing metropolitan area added more than a million people in the past _.
_ races are held around the lake and farmlands.
Bike _ are held around the lake and farmlands.
Bike races _ held around the lake and farmlands.
Bike races are _ around the lake and farmlands.
Bike races are held _ the lake and farmlands.
Bike races are held around _ lake and farmlands.
Bike races are held around the _ and farmlands.
Bike races are held around the lake _ farmlands.
Bike races are held around the lake and _.
_ racist cousin called me last night.
My _ cousin called me last night.
My racist _ called me last night.
My racist cousin _ me last night.
My racist cousin called _ last night.
My racist cousin called me _ night.
My racist cousin called me last _.
_ device is considered to be available if it is not being used by another adult.
A _ is considered to be available if it is not being used by another adult.
A device _ considered to be available if it is not being used by another adult.
A device is _ to be available if it is not being used by another adult.
A device is considered _ be available if it is not being used by another adult.
A device is considered to _ available if it is not being used by another adult.
A device is considered to be _ if it is not being used by another adult.
A device is considered to be available _ it is not being used by another adult.
A device is considered to be available if _ is not being used by another adult.
A device is considered to be available if it _ not being used by another adult.
A device is considered to be available if it is _ being used by another adult.
A device is considered to be available if it is not _ used by another adult.
A device is considered to be available if it is not being _ by another adult.
A device is considered to be available if it is not being used _ another adult.
A device is considered to be available if it is not being used by _ adult.
A device is considered to be available if it is not being used by another _."""

### Process sentences with BERT

In [4]:
# Place [MASK] tokens
MASK = '[MASK]'
sentences = re.sub(r'\b_+\b', '[MASK]', text_sentences).split('\n')

In [5]:
# tokenize input
input_ids = [tokenizer.encode(s, add_special_tokens=True) for s in sentences]

# Find location of MASKS
tok_MASK = tokenizer.convert_tokens_to_ids(MASK)
mask_positions = [s.index(tok_MASK) for s in input_ids] 

# Make all sentence arrays equal length by padding
max_len = max(len(i) for i in input_ids)
padded_input = np.array([i + [0]* (max_len - len(i)) for i in input_ids])

attention_mask = np.where(padded_input != 0, 1, 0)  # Create mask to ignore padding

input = torch.tensor(padded_input)
attention_mask = torch.tensor(attention_mask)

In [6]:
# Get hidden layers
with torch.no_grad():
    last_hidden_states = model(input, attention_mask=attention_mask)

In [7]:
# Get embeddings for the masked word of each sentence
embeddings = [lh[m].numpy() for lh, m in zip(last_hidden_states[0], mask_positions)]

In [8]:
def get_top_predictions(probs, k=5, thres=0.01):
    """
    Print and return top-k predictions for a given probs list.
    Also return predictions above threshold
    """
    # Get top-k tokens
    probs = probs.detach().numpy()
    top_indexes = np.argpartition(probs, -k)[-k:]
    sorted_indexes = top_indexes[np.argsort(-probs[top_indexes])]
    top_tokens = tokenizer.convert_ids_to_tokens(sorted_indexes)
    print(f"Ordered top predicted tokens: {top_tokens}")
    print(f"Ordered top predicted values: {probs[sorted_indexes]}\n")
    
    # Get all tokens above threshold
    high_indexes = np.where(probs > thres)
    high_tokens = tokenizer.convert_ids_to_tokens(high_indexes[0])
    return top_tokens, high_tokens

### Convert last layer logit predictions to probabilities
We can see what are the highest predictions for the blank in each sentence, and their probabilities.

In [9]:
# Convert last hidden state to probs and find tokens
sm = torch.nn.Softmax(dim=0) 
#id_large = tokenizer.convert_tokens_to_ids('large')
all_high_tokens = []
i = 0
for lh, m in zip(last_hidden_states[0], mask_positions):
    print("Sentence:")
    print(sentences[i])
    i += 1
    probs = sm(lh[m])
    #print(f"Probability of 'large': {probs[id_large]}")
    _, high_tokens = get_top_predictions(probs)
    all_high_tokens.append(high_tokens)

Sentence:
[MASK] fat cat ate the mouse.
Ordered top predicted tokens: ['the', 'a', 'that', 'my', 'his']
Ordered top predicted values: [0.9451467  0.03199202 0.00576943 0.00449885 0.00188601]

Sentence:
The [MASK] cat ate the mouse.
Ordered top predicted tokens: ['black', 'cheshire', 'big', 'little', 'fat']
Ordered top predicted values: [0.13267049 0.08640933 0.06516975 0.03538685 0.03100599]

Sentence:
The fat [MASK] ate the mouse.
Ordered top predicted tokens: ['man', 'cat', 'rat', 'dog', 'mouse']
Ordered top predicted values: [0.318098   0.13372828 0.05779354 0.05538516 0.04909774]

Sentence:
The fat cat [MASK] the mouse.
Ordered top predicted tokens: ['.', 'and', ',', 'or', ';']
Ordered top predicted values: [0.5628975  0.32685623 0.05045042 0.00994903 0.0039914 ]

Sentence:
The fat cat ate [MASK] mouse.
Ordered top predicted tokens: ['the', 'a', 'my', 'his', 'her']
Ordered top predicted values: [0.74722546 0.17293347 0.02507927 0.01785319 0.01100375]

Sentence:
The fat cat ate the 

In [10]:
# Cluster embeddings with KMeans
from sklearn.cluster import KMeans, OPTICS, DBSCAN, cluster_optics_dbscan
k = 20
estimator = KMeans(init="k-means++", n_clusters=k, n_jobs=4)
estimator.fit(embeddings)
estimator.labels_

array([ 1,  9, 12, 13,  1, 12, 14, 14, 13,  1, 11, 11,  7,  1,  7, 14, 15,
       17, 14,  1,  6, 12, 14, 14, 15,  1,  6,  8,  1,  1,  6,  8,  2,  2,
        2,  1, 16, 13, 13,  5, 13,  6, 19, 19, 15,  3, 13,  5,  5, 18, 13,
        1,  3,  3, 13, 18, 13, 10, 13, 13, 10, 13, 10,  1,  9,  8, 15,  1,
        1,  7, 17,  8, 17,  4, 17, 17,  4, 17, 17,  0, 17, 17, 15, 13, 17,
       17], dtype=int32)

### Form word categories
Take all words above a threshold from vectors that belong to a cluster to form word categories

In [11]:
word_categories = {}
for cl in range(k):
    cluster_members = np.where(estimator.labels_ == cl)
    word_categories[cl] = sum((all_high_tokens[i] for i in cluster_members[0]), [])
    word_categories[cl] = set(word_categories[cl])
    print(f"Category {cl}:")
    print(", ".join(word_categories[cl]) + "\n")

Category 0:
is

Category 1:
at, your, and, their, from, last, me, one, that, us, you, friday, this, the, for, a, my, in, all, his, her

Category 2:
gonzalez, garcia, lopez, rodriguez, she, gomez, was, born, jewish, castro, sanchez, rivera, diaz, hernandez, perez, he

Category 3:
last, years, century, previous, following, next, ensuing, past, decades, year, same, first, decade, more

Category 4:
dangerous, harmless, intended, illegal, suitable, safe, known, invalid, appropriate, valuable, secure, considered, hazardous, useful, effective, useless, required, safer, harmful, legitimate, permitted, allowed, likely, legal, deemed, helpful, expected, valid, unsafe, not, inappropriate, guaranteed, said, unlikely

Category 5:
3, toronto, 5, 1, florida, 60, million, 15, 2, 12, texas, one, brooklyn, germany, 10, london, 7, 40, six, 8, 25, 6, five, thousand, ten, three, england, california, canada, a, 200, 100, 20, france, philadelphia, two, four, billion, 30, mexico, chicago, 4, 50, 11

Category 